In [ ]:
# I am using google collab as my "IDE" so this is how i set up
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Defining my path
# grabbed my data set off https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page in this case it was Yellow Taxi Feburary 
path = "/content/drive/MyDrive/Colab Notebooks/yellow_tripdata_2023-02.parquet"

In [ ]:
# normally i would use pandas but to work with parquet files we use pyarrow 
# link to documentation -> https://arrow.apache.org/docs/python/parquet.html
import pyarrow.parquet as pq

trips = pq.read_table(path)
trips = trips.to_pandas()

trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [ ]:
trips.info()
# i am looking for the data types in this dataset
# if the datetime columns were of data type 'Object' then you would use pandas .to_datetime function to convert Object -> datetime Dtype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [ ]:
# cleanup (dropping duplicates)
df = trips
df = df.drop_duplicates().reset_index(drop=True)
df = df.dropna()
df['trip_id'] = df.index # giving a trip_id column based on row

<ipython-input-5-1f6d9a7197cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trip_id'] = df.index # giving a trip_id column based on row


In [ ]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,0
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00,1
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00,2
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,3
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00,4


In [ ]:
# replacing the index to suit our Datetime Table in our Data Model
Datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

Datetime['tpep_pickup_datetime'] = Datetime['tpep_pickup_datetime']
Datetime['pickup_hour'] = Datetime['tpep_pickup_datetime'].dt.hour
Datetime['pickup_day'] = Datetime['tpep_pickup_datetime'].dt.day
Datetime['pickup_month'] = Datetime['tpep_pickup_datetime'].dt.month
Datetime['pickup_year'] = Datetime['tpep_pickup_datetime'].dt.year
Datetime['pickup_weekday'] = Datetime['tpep_pickup_datetime'].dt.weekday

Datetime['tpep_dropoff_datetime'] = Datetime['tpep_dropoff_datetime']
Datetime['dropoff_hour'] = Datetime['tpep_dropoff_datetime'].dt.hour
Datetime['dropoff_day'] = Datetime['tpep_dropoff_datetime'].dt.day
Datetime['dropoff_month'] = Datetime['tpep_dropoff_datetime'].dt.month
Datetime['dropoff_year'] = Datetime['tpep_dropoff_datetime'].dt.year
Datetime['dropoff_weekday'] = Datetime['tpep_dropoff_datetime'].dt.weekday

Datetime['datetime_id'] = Datetime.index

Datetime = Datetime[['datetime_id', 'tpep_pickup_datetime', 'pickup_hour', 'pickup_day', 'pickup_month', 'pickup_year', 'pickup_weekday',
                             'tpep_dropoff_datetime', 'dropoff_hour', 'dropoff_day', 'dropoff_month', 'dropoff_year', 'dropoff_weekday']]

# our Datetime model is now complete
Datetime.head()


,datetime_id,tpep_pickup_datetime,pickup_hour,pickup_day,pickup_month,pickup_year,pickup_weekday,tpep_dropoff_datetime,dropoff_hour,dropoff_day,dropoff_month,dropoff_year,dropoff_weekday
0,0,2023-02-01 00:32:53,0,1,2,2023,2,2023-02-01 00:34:34,0,1,2,2023,2
1,1,2023-02-01 00:35:16,0,1,2,2023,2,2023-02-01 00:35:30,0,1,2,2023,2
2,2,2023-02-01 00:35:16,0,1,2,2023,2,2023-02-01 00:35:30,0,1,2,2023,2
3,3,2023-02-01 00:29:33,0,1,2,2023,2,2023-02-01 01:01:38,1,1,2,2023,2
4,4,2023-02-01 00:12:28,0,1,2,2023,2,2023-02-01 00:25:46,0,1,2,2023,2


In [ ]:
# Now we do the same with our other data tables
Passenger_count = df[['passenger_count']].reset_index(drop=True)
Passenger_count['passenger_count_id'] = Passenger_count.index
Passenger_count = Passenger_count[['passenger_count_id','passenger_count']]

trip_distance = df[['trip_distance']].reset_index(drop=True)
trip_distance['trip_distance_id'] = trip_distance.index
trip_distance = trip_distance[['trip_distance_id','trip_distance']]

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

RatecodeID = df[['RatecodeID']].reset_index(drop=True)
RatecodeID['Rate_Code_ID'] = RatecodeID.index
RatecodeID['Ratecode_name'] = RatecodeID['RatecodeID'].map(rate_code_type)
RatecodeID = RatecodeID[['Rate_Code_ID','RatecodeID','Ratecode_name']]

In [ ]:
RatecodeID['RatecodeID'].unique()
# no idea what ratecode 99 is 

array([ 1.,  2.,  5.,  4.,  3., 99.,  6.])

In [ ]:
RatecodeID = RatecodeID[RatecodeID['RatecodeID'] != 99]
# removing the rows with 99 as the identifer as data we dont want
RatecodeID['RatecodeID'].unique()

array([1., 2., 5., 4., 3., 6.])

In [ ]:
Location_Info_T = df[['PULocationID', 'DOLocationID']].reset_index(drop=True)
Location_Info_T['Location_Info'] = Location_Info_T.index
Location_Info_T = Location_Info_T[['Location_Info','PULocationID','DOLocationID']] 

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

Payment_type = df[['payment_type']].reset_index(drop=True)
Payment_type['payment_type_id'] = Payment_type.index
Payment_type['payment_type_name'] = Payment_type['payment_type'].map(payment_type_name)
Payment_type = Payment_type[['payment_type_id','payment_type','payment_type_name']]

In [ ]:
Main_Table = df.merge(Passenger_count, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance, left_on='trip_id', right_on='trip_distance_id') \
             .merge(RatecodeID, left_on='trip_id', right_on='Rate_Code_ID') \
             .merge(Location_Info_T, left_on='trip_id', right_on='Location_Info') \
             .merge(Datetime, left_on='trip_id', right_on='datetime_id') \
             .merge(Payment_type, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'Rate_Code_ID', 'store_and_fwd_flag', 'Location_Info',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount','congestion_surcharge','Airport_fee']]

In [ ]:
Main_Table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,Rate_Code_ID,store_and_fwd_flag,Location_Info,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,0,0,0,0,N,0,0,4.4,3.50,0.5,0.00,0.00,1.0,9.40,2.5,0.00
1,1,2,1,1,1,1,N,1,1,-3.0,-1.00,-0.5,0.00,0.00,-1.0,-5.50,0.0,0.00
2,2,2,2,2,2,2,N,2,2,3.0,1.00,0.5,0.00,0.00,1.0,5.50,0.0,0.00
3,3,1,3,3,3,3,N,3,3,70.9,2.25,0.5,0.00,0.00,1.0,74.65,0.0,1.25
4,4,2,4,4,4,4,N,4,4,17.0,1.00,0.5,3.30,0.00,1.0,25.30,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824096,2837133,2,2837133,2837133,2837133,2837133,N,2837133,2837133,10.0,1.00,0.5,3.00,0.00,1.0,18.00,2.5,0.00
2824097,2837134,2,2837134,2837134,2837134,2837134,N,2837134,2837134,10.7,1.00,0.5,3.14,0.00,1.0,18.84,2.5,0.00
2824098,2837135,2,2837135,2837135,2837135,2837135,N,2837135,2837135,16.3,1.00,0.5,4.26,0.00,1.0,25.56,2.5,0.00
2824099,2837136,2,2837136,2837136,2837136,2837136,N,2837136,2837136,37.3,1.00,0.5,0.00,6.55,1.0,48.85,2.5,0.00


In [ ]:
# I am saving my cleaned data into a csv file for my next step which is creating a data pipeline in Mage AI
Main_Table.to_csv('/content/drive/MyDrive/Data/cleaned_uber_data.csv', index=False)